In [1]:
import pandas as pd
from bs4 import BeautifulSoup
# import urllib.request
# import urllib.parse
# import urllib.error
import ssl
import json
# import ast
# import os
from urllib.request import Request, urlopen
import random
import time

## URLs

##### This section generates all URLs that will be used to scrape song lyrics from genius.com. When adding songs, remove any punctuation and write all lowercase

In [69]:
base_url = 'https://genius.com/'
end_url = '-lyrics'

# list of songs
billie_eilish_songs = ['copycat', 'idontwannabeyouanymore', 'my boy', 'watch', 'party favor',
                       'bellyache', 'ocean eyes', 'hostage', 'and vince staples burn', 
                       'everything i wanted', 'bad guy', 'xanny', 'you should see me in a crown',
                       'all the good girls go to hell', 'wish you were gay', 
                       'when the partys over', '8', 'my strange addiction', 'bury a friend',
                       'ilomilo', 'listen before i go', 'i love you', 'goodbye',
                       'everything i wanted']

# did 'velvet mood' and 'recital' so far
billie_holiday_songs = ['prelude to a kiss', 'when your lover has gone', 
                        'please dont talk about me when im gone', 'nice work if you can get it',
                        'ive got a right to sing the blues', 'whats new', 'i hadnt anyone till you',
                        'everything i have is yours', 'if the moon turns green', 'remember',
                        'autumn in new york', 'my man', 'lover come back to me', 'stormy weather',
                        'yesterdays', 'hes funny that way', 'i cant face the music',
                        'how deep is the ocean', 'what a little moonlight can do', 'i cried for you']

# did 'some gave all' so far
billy_ray_cyrus_songs = ['couldve been me', 'achy breaky heart', 'shes not cryin anymore',
                         'wherm i gonna live', 'these boots are made for walkin', 
                         'someday somewhere somehow', 'never thought id fall in love with you',
                         'aint no good goodbye', 'im so miserable', 'some gave all']

# did 'cold spring harbor' so far
# nocturne is not on this list bc it doesn't have proper lyrics
billy_joel_songs = ['shes got a way', 'you can make me free', 'everybody loves you now', 
                    'why judy why', 'falling of the rain', 'turn around', 'you look so good to me',
                    'tomorrow is today', 'got to begin again']

# did 'billy idol' album so far
billy_idol_songs = ['come on come on', 'white wedding', 'hot in the city', 'dead on arrival',
                    'nobodys business', 'love calling', 'hole in the wall', 'shooting stars',
                    'its so cruel', 'congo man']

# did 'billy_currington' album so far
billy_currington_songs = ['i got a feelin', 'thats just me', 'hangin around', 'off my rocker',
                          'walk a little straighter', 'where the girls are', 'time with you',
                          'when she gets close to me', 'growin up down there', 'next time', 
                          'aint what it used to be']

# did 'tale of the tape' album so far
billy_squier_songs = ['the big beat', 'calley oh', 'rich kid', 'like im lovin you',
                      'who knows what a love can do', 'you should be high love', 
                      'whos your boyfriend', 'the musics all right', 'young girls']

billy_graham_songs = []
lil_nas_x_songs = ['old town road remix']

# create url slugs
billie_eilish_slugs = (['Billie-eilish' + '-' + '-'.join(song.split(' ')) 
                        for song in billie_eilish_songs])
billie_holiday_slugs = (['Billie-holiday' + '-' + '-'.join(song.split(' ')) 
                        for song in billie_holiday_songs])
billy_ray_cyrus_slugs = (['Billy-ray-cyrus' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_ray_cyrus_songs])
billy_joel_slugs = (['Billy-joel' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_joel_songs])
billy_idol_slugs = (['Billy-idol' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_idol_songs])
billy_currington_slugs = (['Billy-currington' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_currington_songs])
billy_squier_slugs = (['Billy-squier' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_squier_songs])
billy_graham_slugs = (['Billy-graham' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_graham_songs])
lil_nas_x_slugs = (['Lil-nas-x' + '-' + '-'.join(song.split(' ')) 
                        for song in lil_nas_x_songs])

# list of all the slugs
slug_list = (billie_eilish_slugs + billie_holiday_slugs + billy_ray_cyrus_slugs
             + billy_joel_slugs + billy_idol_slugs + billy_graham_slugs
             + billy_currington_slugs + billy_squier_slugs
             + lil_nas_x_slugs)

################################################
################ TESTING #######################
####### COMMENT THIS OUT TO DO ALL URLS ########
################################################
# slug_list = (billy_joel_slugs + billy_idol_slugs + billy_graham_slugs
#              + billy_currington_slugs + billy_squier_slugs)

# generate urls
url_list = [base_url + slug + end_url for slug in slug_list]
print('There are ', len(url_list), 'urls in the list')

There are  94 urls in the list


## Scraping

In [61]:
def scrape_songs(url_list):
    """This function takes in a list of URLs from genius.com,
        scrapes the lyrics and title, and returns a dataframe 
        of [artist, title, lyrics]"""
    
    counter = 1
    scrape_list = []
    
    # iterate through all URLs and scrape 
    for url in url_list:
        
        try:
            print(counter, ' starting ', url)

            # ignore SSL certificate errors
            ctx = ssl.create_default_context()
            ctx.check_hostname = False
            ctx.verify_mode = ssl.CERT_NONE

            # make scrape request appear as browser
            req = Request(url, headers = {'User-Agent' : 'Mozilla/5.0'})
            webpage = urlopen(req).read()

            soup = BeautifulSoup(webpage, 'html.parser')
            html = soup.prettify('utf-8')
            song_json = {}
            song_json['lyrics'] = []

            # extract title of the song
            for title in soup.findAll('title'):
                song_json['title'] = title.text.strip()

            # extract the lyrics of the song
            for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
                song_json['lyrics'].append(div.text.strip().split('\n'))

            scrape_list.append(song_json)
            time.sleep(random.randint(1,5))
        
        except:
            print('error with ', url)
            continue
        
        counter += 1

    
    # create dataframe of scraped songs
    df = pd.DataFrame(scrape_list)

    # fix string formatting issue
    # NOTE: the '–' character used in titles column is not a normal dash
    # I had to print a title out and use that dash for it to recognize the 
    # character. Replacing it with a normal - to avoid dealing with it.
    # They look the same but they're not ¯\_(ツ)_/¯
    df['title'] = [title.replace('–','-') for title in df['title']]

    # add artist column
    df['artist'] = [title.split('-')[0] for title in df['title']]
    
    # clean title column
    df['title'] = [title.replace('Lyrics','-') for title in df['title']]
    df['title'] = [title.split('-')[1] for title in df['title']]
    
    return df


df = scrape_songs(url_list)

1  starting  https://genius.com/Billie-eilish-copycat-lyrics
2  starting  https://genius.com/Billie-eilish-idontwannabeyouanymore-lyrics
3  starting  https://genius.com/Billie-eilish-my-boy-lyrics
4  starting  https://genius.com/Billie-eilish-watch-lyrics
5  starting  https://genius.com/Billie-eilish-party-favor-lyrics
6  starting  https://genius.com/Billie-eilish-bellyache-lyrics
7  starting  https://genius.com/Billie-eilish-ocean-eyes-lyrics
8  starting  https://genius.com/Billie-eilish-hostage-lyrics
9  starting  https://genius.com/Billie-eilish-and-vince-staples-burn-lyrics
10  starting  https://genius.com/Billie-eilish-everything-i-wanted-lyrics
11  starting  https://genius.com/Billie-eilish-bad-guy-lyrics
12  starting  https://genius.com/Billie-eilish-xanny-lyrics
13  starting  https://genius.com/Billie-eilish-you-should-see-me-in-a-crown-lyrics
14  starting  https://genius.com/Billie-eilish-all-the-good-girls-go-to-hell-lyrics
15  starting  https://genius.com/Billie-eilish-wish-

In [62]:
df

,lyrics,title,artist
0,"[[[Verse 1], Don't be cautious, don't be kind,...",COPYCAT,Billie Eilish
1,"[[[Verse 1], Don't be that way, Fall apart twi...",​idontwannabeyouanymore,Billie Eilish
2,"[[[Intro], (Three and four and), Ba-ba-da, ba-...",​my boy,Billie Eilish
3,"[[[Verse 1], Lips meet teeth and tongue, My he...",​watch,Billie Eilish
4,"[[[Intro], Hey, leave a message, , [Verse 1], ...",​​party favor,Billie Eilish
5,"[[[Intro], Mind, Mind, , [Verse 1], Sittin' al...",​bellyache,Billie Eilish
6,"[[[Verse 1], I've been watchin' you for some t...",​​ocean eyes,Billie Eilish
7,"[[[Verse 1], I wanna be alone, Alone with you,...",​hostage,Billie Eilish
8,"[[[Verse 1: Billie Eilish], Lips meet teeth an...",​&burn,Billie Eilish & Vince Staples
9,"[[[Verse 1], I had a dream, I got everything I...",​everything i wanted,Billie Eilish


In [63]:
def clean_lyrics(df):
    """This function takes a dataframe of lyrics and removes any unwanted
       characters and returns the dataframe"""
    
    df = df.copy()
    # turn lyrics column into a string
    df['lyrics'] = [str(lyrics[0]) for lyrics in df['lyrics']]

    # custom words and characters to remove
    stop_words = ['Verse 1', 'Verse 2', 'Verse 3', 'Verse 4', 'Chorus', 
                  'Bridge', ',,', 'Intro:', 'Intro',
                  'Billie Eilish', 'Billie Holiday', 'Billy Rae Cyrus',
                  'Billy Joel', 'Billy Idol', 'Billy Graham', 
                  'Vince Staples',
                  '\\n', '{', '}', '[', ']', 'lyrics', 'title', '"', 'genius', "'',"]
    # text = text.replace("\", '')
    
    for word in stop_words:
        df['lyrics'] = [lyrics.replace(word, '') for lyrics in df['lyrics']]
        
    return df

In [64]:
clean_df = clean_lyrics(df)
clean_df

,lyrics,title,artist
0,"Don't be cautious, don't be kind, You committ...",COPYCAT,Billie Eilish
1,"Don't be that way, 'Fall apart twice a day', ...",​idontwannabeyouanymore,Billie Eilish
2,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b...",​my boy,Billie Eilish
3,"'Lips meet teeth and tongue', 'My heart skips...",​watch,Billie Eilish
4,"'Hey, leave a message', 'Hey, call me back ...",​​party favor,Billie Eilish
5,"'Mind', 'Mind', Sittin' all alone, 'Mouth f...",​bellyache,Billie Eilish
6,"I've been watchin' you for some time, Can't s...",​​ocean eyes,Billie Eilish
7,"'I wanna be alone', 'Alone with you, does tha...",​hostage,Billie Eilish
8,"': ', 'Lips meet teeth and tongue', My heart s...",​&burn,Billie Eilish & Vince Staples
9,"'I had a dream', 'I got everything I wanted',...",​everything i wanted,Billie Eilish


## Testing

In [65]:
print(clean_df.lyrics.loc[8])
print(type(clean_df.lyrics.loc[8]))

': ', 'Lips meet teeth and tongue', My heart skips eight beats at once (That's better), 'If we were meant to be, we would have been by now', 'See what you wanna see, all I see is him right now', 'H-h-him right now',  ': ', I'll sit and watch your car burn, 'With the fire that you started in me', 'But you never came back to ask it out', 'Go ahead and watch my heart burn', 'With the fire that you started in me', But I'll never let you back to put it out, '(Thanks)',  ': ', 'Your love feels so fake', My demands aren't high to make, 'If I could get to sleep, I would have slept by now (Ahh)', 'Your lies will never keep, I think you need to blow them out', '(B-b-blow them out)',  ': ', I'll sit and watch your car burn, 'With the fire that you started in me', 'But you never came back to ask it out', 'Go ahead and watch my heart burn', 'With the fire that you started in me', But I'll never let you back to put it out,  ': ', '7-4-2008', 'I still remember that', 'Heaven sent a present my way', I

## Export

In [66]:
clean_df.to_csv('scraped_lyrics.csv', index=False)

In [67]:
pd.read_csv('scraped_lyrics.csv')

,lyrics,title,artist
0,"Don't be cautious, don't be kind, You committ...",COPYCAT,Billie Eilish
1,"Don't be that way, 'Fall apart twice a day', ...",​idontwannabeyouanymore,Billie Eilish
2,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b...",​my boy,Billie Eilish
3,"'Lips meet teeth and tongue', 'My heart skips...",​watch,Billie Eilish
4,"'Hey, leave a message', 'Hey, call me back ...",​​party favor,Billie Eilish
5,"'Mind', 'Mind', Sittin' all alone, 'Mouth f...",​bellyache,Billie Eilish
6,"I've been watchin' you for some time, Can't s...",​​ocean eyes,Billie Eilish
7,"'I wanna be alone', 'Alone with you, does tha...",​hostage,Billie Eilish
8,"': ', 'Lips meet teeth and tongue', My heart s...",​&burn,Billie Eilish & Vince Staples
9,"'I had a dream', 'I got everything I wanted',...",​everything i wanted,Billie Eilish
